### Talking Bot with Seq2Seq RNN

本节参考 [斗大的熊猫](http://blog.topspeedsnail.com/archives/10735) ，使用 Seq2Seq RNN 模型在中文对话语料上进行 RNN 学习和生成

代码参考

- [使用深度学习打造智能聊天机器人](http://blog.csdn.net/malefactor/article/details/51901115)
- [脑洞大开：基于美剧字幕的聊天语料库建设方案](http://www.shareditor.com/blogshow/?blogId=105)
- [Seq2Seq](https://www.tensorflow.org/versions/r0.12/tutorials/seq2seq/index.html)

语料来自 [中文对白语料](https://github.com/rustch3n/dgk_lost_conv)

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'0.9.0'

In [3]:
import random
import codecs
from collections import defaultdict as dd

"""
文件格式
E
M 畹/华/吾/侄/
M 你/接/到/这/封/信/的/时/候/
M 不/知/道/大/伯/还/在/不/在/人/世/了/
E
M 咱/们/梅/家/从/你/爷/爷/起/
M 就/一/直/小/心/翼/翼/地/唱/戏/
..........
M 就/因/为/没/穿/红/让/人/赏/咱/一/纸/枷/锁/
M 爷/您/别/给/我/戴/这/纸/枷/锁/呀/
E
..........
"""
datafile = './data/shooter/dgk_shooter_min.conv'

# 特殊标记，用来填充标记对话
PAD = "__PAD__"
GO = "__GO__"
EOS = "__EOS__"  # 对话结束
UNK = "__UNK__"  # 标记未出现在词汇表中的字符
START_VOCABULART = [PAD, GO, EOS, UNK]    # 在词典中居前 4 位
# 在词典中位置
PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3

VOCAB_SIZE = 5000
TEST_SIZE = 8000

In [4]:
def get_convs(datafile):
    """
    返回对话数组，每个对话指两个 E 之间的部分；结果类似下面
    [ ['畹华吾侄', '你接到这封信的时候', '不知道大伯还在不在人世了'],
      ['咱们梅家从你爷爷起', '就一直小心翼翼地唱戏', ......],
      ......
    ]
    """
    convs = []   # store conversation
    with codecs.open(datafile, 'r', 'utf-8') as fp:
        conv = []
        for line in fp:
            line = line.strip().replace('/', '')
            if line == '':
                continue
            # end of conversation
            if line[0] == 'E':
                if conv:
                    convs.append(conv)
                conv = []
            elif line[0] == 'M':
                conv.append(line.split(' ')[1])
    print "total conversations: {}".format(len(convs))
    return convs


In [5]:
def convs_to_qafile(convs):
    """
    把对话拆分为问答
    这个分法比较简单粗暴，故此最后结果也不会非常之好
    """
    questions = []
    answers = []
    for conv in convs:
        # 如果对话只有一句，那么无法拆分
        if len(conv) == 1:
            continue
        # 如果奇数对话，那么转为偶数，扔掉最后一句
        if len(conv) % 2 != 0:
            conv = conv[: -1]
        for i, sentence in enumerate(conv):
            if i % 2 == 0:
                questions.append(sentence)
            else:
                answers.append(sentence)
    print "Total questions/answers: {}".format(len(questions))

    train_enc = codecs.open('./data/shooter/train.enc', 'w', 'utf-8')
    train_dec = codecs.open('./data/shooter/train.dec', 'w', 'utf-8')
    test_enc = codecs.open('./data/shooter/test.enc', 'w', 'utf-8')
    test_dec = codecs.open('./data/shooter/test.dec', 'w', 'utf-8')

    vocab_enc = codecs.open('./data/shooter/vocab.enc', 'w', 'utf-8')
    vocab_dec = codecs.open('./data/shooter/vocab.dec', 'w', 'utf-8')
    # questions 和 answers 各自做一个词典，而不是公用一个词典
    words_enc = dd(int)
    words_dec = dd(int)

    # 取出 TEST_SIZE 个作为测试集
    test_index = random.sample([i for i in range(len(questions))], TEST_SIZE)
    for i, question in enumerate(questions):
        # 分别统计 q / a 的词频
        for w in question:
            words_enc[w] += 1
        for w in answers[i]:
            words_dec[w] += 1

        # 把 q / a 划分到 train / test 集
        if i in test_index:
            test_enc.write(question + '\n')
            test_dec.write(answers[i] + '\n')
        else:
            train_enc.write(question + '\n')
            train_dec.write(answers[i] + '\n')
        if i % 1000 == 0:
            print "{} qa pairs processed".format(i)
    train_enc.close()
    train_dec.close()
    test_enc.close()
    test_dec.close()

    for words, vocabfp in [(words_enc, vocab_enc), (words_dec, vocab_dec)]:
        # 把字符按出现次数倒序排列，并在前面加上特殊字符
        ordered_vocab = START_VOCABULART + sorted(words, key=words.get, reverse=True)
        # 取前 VOCAB_SIZE 个常见字，这里其实可以做更多的数据梳理
        ordered_vocab = ordered_vocab[: VOCAB_SIZE]
        for w in ordered_vocab:
            vocabfp.write(w + '\n')

    vocab_enc.close()
    vocab_dec.close()

In [6]:
def convert_to_vector(infile, vocabfile, outfile):
    vocabs = []
    with codecs.open(vocabfile, 'r', 'utf-8') as f:
        for line in f:
            vocabs.append(line.strip())
    vocabs = dict([(x, y) for (y, x) in enumerate(vocabs)])

    with open(outfile, 'w') as outfp:
        with codecs.open(infile, 'r', 'utf-8') as infp:
            for line in infp:
                # 把每行句子转为矢量保存
                vec = []
                for w in line.strip():
                    vec.append(vocabs.get(w, UNK_ID))
                # 索引之间空格相隔
                outfp.write(" ".join([str(idx) for idx in vec]) + '\n')


In [ ]:
convs_to_qafile(get_convs(datafile))

total conversations: 762716
```
$ wc -l data/shooter/*
  4268084 data/shooter/dgk_shooter_min.conv
     8000 data/shooter/test.dec
     8000 data/shooter/test.enc
  1538628 data/shooter/train.dec
  1538628 data/shooter/train.enc
     5000 data/shooter/vocab.dec
     5000 data/shooter/vocab.enc
  7371340 total

```
看到词典都是 5000 个字符；测试集的问答都是 8000 句；训练集的问答都是 1538628 句

In [7]:
print "vecterize train encode file"
convert_to_vector("data/shooter/train.enc", "data/shooter/vocab.enc", 'data/shooter/train_encode.vec')
print "vecterize train decode file"
convert_to_vector("data/shooter/train.dec", "data/shooter/vocab.dec", 'data/shooter/train_decode.vec')
print "vecterize test encode file" 
convert_to_vector("data/shooter/test.enc", "data/shooter/vocab.enc", 'data/shooter/test_encode.vec')
print "vecterize test decode file"
convert_to_vector("data/shooter/test.dec", "data/shooter/vocab.dec", 'data/shooter/test_decode.vec')

vecterize train encode file
vecterize train decode file
vecterize test encode file
vecterize test decode file


```
$ wc -l data/shooter/*.vec
    8000 test_decode.vec
    8000 test_encode.vec
 1538628 train_decode.vec
 1538628 train_encode.vec
```

In [4]:
buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]
layer_size = 256
num_layers = 3
batch_size = 64

def s2s_read_data(enc_path, dec_path, max_size=None):
    """ 每个桶装对应长度的数据 """
    data_set = [[] for _ in buckets]
    with tf.gfile.GFile(enc_path, mode='r') as ef:
        with tf.gfile.GFile(dec_path, mode='r') as df:
            source, target = ef.readline(), df.readline()
            counter = 0
            while source and target and (not max_size or counter < max_size):
                counter += 1
                source_ids = [int(x) for x in source.split()]
                target_ids = [int(x) for x in target.split()]
                # 结束标志 EOS_ID
                target_ids.append(EOS_ID)
                for bucket_id, (source_size, target_size) in enumerate(buckets):
                    if len(source_ids) < source_size and len(target_ids) < target_size:
                        data_set[bucket_id].append([source_ids, target_ids])
                        break
                source, target = ef.readline(), df.readline()
    return data_set


In [14]:
from tensorflow.models.rnn.translate import seq2seq_model

model = seq2seq_model.Seq2SeqModel(VOCAB_SIZE, VOCAB_SIZE, buckets=buckets,
                                   size=layer_size, num_layers=num_layers, max_gradient_norm=5.0, batch_size=batch_size,
                                   learning_rate=0.5, learning_rate_decay_factor=0.97, forward_only=False)


In [ ]:
# config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC'
from __future__ import division
import math

sess = tf.Session()
# 不要在当前目录下找 checkpoint，他会找全部 ckpt-xx 结尾的 checkpoint 文件，可能会读取其他项目的 checkpoint
ckpt = tf.train.get_checkpoint_state('data/shooter/')
if ckpt != None:
    print(ckpt.model_checkpoint_path)
    model.saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.initialize_all_variables())

train_set = s2s_read_data('data/shooter/train_encode.vec', 'data/shooter/train_decode.vec')
test_set = s2s_read_data('data/shooter/test_encode.vec', 'data/shooter/test_decode.vec')

# say [3, 5, 2]
train_bucket_sizes = [len(train_set[i]) for i in range(len(buckets))]
# ==> 3 + 5 + 2 = 10
train_total_size = sum(train_bucket_sizes)
# ==> [0.3, 0.8, 1.0]
train_buckets_scale = [sum(train_bucket_sizes[: i + 1]) / train_total_size for i in range(len(train_bucket_sizes))]
print "train_bucket_sizes: {}".format(train_bucket_sizes)
print "train_total_size: {}".format(train_total_size)
print "train_buckets_scale: {}".format(train_buckets_scale)

loss = 0.0
total_step = 0
previous_losses = []
# 持续训练
print "start training ..."
while True:
    # 随机出一个 0~1 的小数
    random_number_01 = np.random.random_sample()
    # 找到这个随机数在 scale 中的位置，返回这个位置，也就是说，这里根据随机数找到一个桶
    bucket_id = min([i for i in range(len(train_buckets_scale)) if train_buckets_scale[i] > random_number_01])
    # 从对应的桶 id 中，读取一个 batch 的训练 enc & dec 文本，以及权重，这个由 seq2seq model 提供函数实现
    encoder_inputs, decoder_inputs, target_weights = model.get_batch(train_set, bucket_id)
    # 进行训练，最后 forward_only 参数为 False
    _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, False)

    loss += step_loss / 100
    total_step += 1

    if total_step % 100 == 0:
        print "----- total_step: {} -----".format(total_step)
        print "global step, learning rate & loss: {}, {}, {}".format(model.global_step.eval(session=sess), model.learning_rate.eval(session=sess), loss)

        # 如果模型没有得到提升，减小learning rate
        if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
            sess.run(model.learning_rate_decay_op)
        previous_losses.append(loss)

        checkpoint_path = "data/shooter/chatbot_seq2seq.ckpt"
        model.saver.save(sess, checkpoint_path, global_step=model.global_step)
        loss = 0.0

        # 通过测试数据评估
        for bid in range(len(buckets)):
            if len(test_set[bid]) == 0:
                continue
            encoder_inputs, decoder_inputs, target_weights = model.get_batch(test_set, bucket_id)
            # 这里最后一个参数 forward_only 是 True，而前面训练时为 False
            _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, True)
            eval_ppx = math.exp(eval_loss) if eval_loss < 300 else float('inf')
            print "test bucket id:  {}     eval_ppx: {}".format(bid, eval_ppx)


train_bucket_sizes: [274762, 664553, 575636, 23211]
train_total_size: 1538162
train_buckets_scale: [0.17863007927643512, 0.6106736481592966, 0.9849099119598586, 1.0]
start training ...
----- total_step: 100 -----
global step, learning rate & loss: 100, 0.5, 7.01408500671
test bucket id:  0     eval_ppx: 257.649460128
test bucket id:  1     eval_ppx: 238.252258232
test bucket id:  2     eval_ppx: 214.257065068
test bucket id:  3     eval_ppx: 213.714537918
----- total_step: 200 -----
global step, learning rate & loss: 200, 0.5, 5.72609845161
test bucket id:  0     eval_ppx: 160.29752383
test bucket id:  1     eval_ppx: 149.999082452
test bucket id:  2     eval_ppx: 139.566065155
test bucket id:  3     eval_ppx: 132.77818862
----- total_step: 300 -----
global step, learning rate & loss: 300, 0.5, 5.42118005276
test bucket id:  0     eval_ppx: 174.560686236
test bucket id:  1     eval_ppx: 180.965441786
test bucket id:  2     eval_ppx: 166.261146333
test bucket id:  3     eval_ppx: 195.68

**开始使用生成的模型来生成对话**

In [5]:
import sys
import codecs
from tensorflow.models.rnn.translate import seq2seq_model

def read_vocabulary(vocabfile):
    vocabs = []
    with codecs.open(vocabfile, 'r', 'utf-8') as f:
        for line in f:
            vocabs.append(line.strip())

    vocab_dict = dict([(x, y) for (y, x) in enumerate(vocabs)])
    return vocab_dict, vocabs


vocab_enc, _ = read_vocabulary("data/shooter/vocab.enc")
_, vocab_dec = read_vocabulary("data/shooter/vocab.dec")
# batch size 在生成过程中，设置为 1，一问一答！
batch_size = 1

# 训练时， forward_only 为 False
model = seq2seq_model.Seq2SeqModel(VOCAB_SIZE, VOCAB_SIZE, buckets=buckets, size=layer_size, num_layers=num_layers, max_gradient_norm= 5.0,
                                   batch_size=batch_size, learning_rate=0.5, learning_rate_decay_factor=0.99, forward_only=True)


In [6]:
sess = tf.Session()
ckpt = tf.train.get_checkpoint_state('data/shooter')
if ckpt != None:
    print(ckpt.model_checkpoint_path)
    model.saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("Failed to find model file")
    sys.exit(1)


data/shooter/chatbot_seq2seq.ckpt-4900


In [ ]:

while True:
    input_string = raw_input('Me > ').decode('utf-8')
    if input_string == 'quit':
        sys.exit(0)

    input_string_vec = []
    for w in input_string.strip():
        input_string_vec.append(vocab_enc.get(w, UNK_ID))
    bucket_id = min([i for i in range(len(buckets)) if buckets[i][0] > len(input_string_vec)])
    # 从第一个参数中取 batch，而第一个参数只是一个 bucket id 对应的数据集，数据集中也只有一个元素，而这个元素只有 encoder 部分，没有 decoder
    # 故此，很显然取到的 batch 只有 input_string_vec 一条 encoder
    encoder_inputs, decoder_inputs, target_weights = model.get_batch({bucket_id: [(input_string_vec, [])]}, bucket_id)
    # 最后一个参数 forward_only 为 True
    _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, True)
    # 取 logits 中每个 logit 所最大化对应的词典中的字；就是说，返回的 output_logits 是一个数组，对应一句话
    # 其中每个元素为一个 logit，对应句子中的一个字，维度为 VOCAB_SIZE，对应词典中每个字的概率
    # 这个对返回的句子每个字取最可能的值
    outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]
    if EOS_ID in outputs:
        outputs = outputs[: outputs.index(EOS_ID)]

    response = "".join([tf.compat.as_str(vocab_dec[output]) for output in outputs])
    print('AI > ' + response)


ipython notebook 似乎对 raw_input 和 unicode 显示支持的不是很好，那么在 shell 下运行结果如下：
```
data/shooter/chatbot_seq2seq.ckpt-4900
Me > 嘿
AI > 我
Me > 你好
AI > 我
Me > 你只会说我？
AI > 我是我
Me > 你到底是谁
AI > 我是我
Me > 今天天气不错
AI > 我是我
Me > 听风和日丽的
AI > 我是
Me > 你疯了？
AI > 我是
Me > 谢谢你
AI > 我
Me > 昨天听说有一场不错的演唱会呢
AI > 我们是我的
Me > 你除了说我还会说什么呢？
AI > 我是我
```

训练的轮数还不多，看到效果也不是很好哈